In [1]:
import pandas as pd, numpy as np
pd.options.display.float_format = '{:,.4f}'.format
import plotly.express as px; from plotly import graph_objects as go
import datetime as dt; ts = dt.datetime.now(); ts_str = ts.strftime("%Y_%m_%d_%H_%M_%S"); print(ts_str)

2020_09_08_14_29_00


In [2]:
sql = 'SELECT * FROM [MPlanProd].[dbo].[View_Transaction_SFT_6049]'
con = (
    r'DRIVER=ODBC Driver 17 for SQL Server;'
    r'SERVER=172.31.228.90;'
    r'DATABASE=MPlanProd;'
    r'Trusted_Connection=no;'
    r'Uid=DataEng;'
    r'Pwd=Data@2020;'
)

In [3]:
%%time
import pyodbc
cxn = pyodbc.connect(con)
df1 = pd.read_sql(sql, cxn)
print(df1.shape, ', '.join(df1.columns))

(284594, 19) SAPCode, SiteName, WorkCentreName, Sloc, Day, Time, SupplierName, PlateLicense, TruckType, Ticket_no, Size_inch, InboundWeight, MillWeight, OutboundWeight, Purchase_price_thb_ton, Price_ton, Month, Year, TransactionType
Wall time: 44.9 s


# Cleansing

In [4]:
df = df1.copy()
df = df.loc[df.Size_inch=='ยูคาไม้ท่อน 2.0 นิ้ว']
df = df.loc[~df.SiteName.isin(['สามง่าม','วังวน','ตาก','บางกระทุ่ม', 'CC6'])]
df.loc[df.SiteName=='โรงสับไม้กำแพงเพชร', 'Sloc'] = 'CH11'
df.loc[df.SiteName=='โรงสับไม้ชุมพวง', 'Sloc'] = 'CH12'
df.loc[df.SiteName=='โรงสับไม้พระยืน', 'Sloc'] = 'CH13'
df['Zone'] = df['Sloc'].str[:2]

In [ ]:
col_keep = ['Zone', 'Sloc', 'Year', 'Month', 'Day', 'MillWeight', 'Price_ton', 'Purchase_price_thb_ton']
df[col_keep].to_csv('View_Transaction_SFT_6049_'+ts_str+'.csv', index=False)
#df = pd.read_csv('View_Transaction_SFT_6049_'+ts_str+'.csv', parse_dates=True)
#df['dt'] = pd.to_datetime(df['dt'].astype(str) + ' ' + df['Time'].astype(str))

In [5]:
ck=colkeep=['SiteName','Ticket_no'] # WorkCentreName, 'Time'
df = df[ck]

In [ ]:
#df.SiteName.value_counts()#.sum()

In [ ]:
#df.Sloc.value_counts()#.sum()

In [26]:
sloc = dict(df.groupby('SiteName')['Sloc'].max())
sloc = dict([(value, key) for key, value in sloc.items()])
zone = df.groupby('Zone').agg({'Sloc': lambda x: list(np.unique(x))})['Sloc'].to_dict()

# Cumulative

In [34]:
ym, ymi, ymc, ymp = {}, {}, {}, {}
months = [dt.date(2008, i, 1).strftime('%b') for i in range(1,13)]

In [35]:
for s in ['all']+list(zone.keys())+list(sloc.keys()):
    if s == 'all':
        dfs = df
    elif len(s)==2:
        dfs = df.loc[df.Zone==s]
    else:
        dfs = df.loc[df.Sloc==s]
    ym[s], ymi[s], ymc[s] = {}, {}, {}
    for t in ['Purchase_price_thb_ton','MillWeight']:
        ym[s][t] = dfs.groupby(['Year','Month'])[t].sum().reset_index()
        ymi[s][t]= ym[s][t][ym[s][t].Year>=2016].reset_index(drop=True)
        ymc[s][t]= ymi[s][t].groupby('Year').cumsum().merge(ymi[s][t], left_index=True,right_index=True,suffixes=('_c',''))
    ymp[s] = ymc[s]['Purchase_price_thb_ton'].merge(ymc[s]['MillWeight'])
    ymp[s]['Price']  = ymp[s]['Purchase_price_thb_ton'] / ymp[s]['MillWeight']
    ymp[s]['Price_c']= ymp[s]['Purchase_price_thb_ton_c'] / ymp[s]['MillWeight_c']
    ymp[s].Year = ymp[s].Year.astype('category')
    ymp[s].Month = ymp[s].Month.astype('category')

In [36]:
for t in ['Purchase_price_thb_ton','MillWeight']:
    if t == 'MillWeight': cols = ['MillWeight', 'MillWeight_c']
    else: cols = ['Purchase_price_thb_ton', 'Purchase_price_thb_ton_c']
    for col in cols:
        fig = go.Figure()
        for year in ymc[s][t].Year.unique():
            Y  = ymc[s][t].loc[ymc[s][t].Year==year]
            fig.add_trace(go.Scatter(x=months, y=Y[col], mode="markers+text+lines", textposition="middle right",
                                     text=['' for i in range(len(Y)-1)]+[str(year)]))
        fig.update_layout(showlegend=False, title=col)
        fig.show()

In [37]:
for col in ['Price', 'Price_c']:
    fig = go.Figure()
    for year in ymp[s].Year.unique():
        Y  = ymp[s].loc[ymp[s].Year==year]
        fig.add_trace(go.Scatter(x=months, y=Y[col],
                                 mode="markers+text+lines", textposition="middle right",
                                 text=['' for i in range(len(Y)-1)]+[str(year)]))
    fig.update_layout(showlegend=False, title=col)
    fig.show()

# Supply Estimation

In [44]:
ymp['all']#.Month.isin([10, '11', '12', '1', '2', '3', '4'])

,Month_c,Purchase_price_thb_ton_c,Year,Month,Purchase_price_thb_ton,MillWeight_c,MillWeight,Price,Price_c
0,1,"17,713,362.7000",2016,1,"17,713,362.7000","16,233.3500","16,233.3500","1,091.1711","1,091.1711"
1,3,"38,555,740.1000",2016,2,"20,842,377.4000","35,359.1700","19,125.8200","1,089.7508","1,090.4029"
2,6,"59,742,993.3000",2016,3,"21,187,253.2000","54,382.5000","19,023.3300","1,113.7510","1,098.5702"
3,10,"75,414,672.5000",2016,4,"15,671,679.2000","68,851.1600","14,468.6600","1,083.1466","1,095.3290"
4,15,"89,028,073.6000",2016,5,"13,613,401.1000","85,337.8000","16,486.6400",825.7232,"1,043.2431"
5,21,"97,201,599.4000",2016,6,"8,173,525.8000","97,595.6200","12,257.8200",666.8009,995.9627
6,28,"105,310,921.6400",2016,7,"8,109,322.2400","108,733.7700","11,138.1500",728.0672,968.5208
7,36,"110,564,474.2300",2016,8,"5,253,552.5900","119,445.6100","10,711.8400",490.4435,925.6470
8,45,"119,080,354.6100",2016,9,"8,515,880.3800","131,030.6600","11,585.0500",735.0750,908.7976
9,55,"131,128,776.2200",2016,10,"12,048,421.6100","141,839.1900","10,808.5300","1,114.7142",924.4890


# Export

In [ ]:
wp = {}
for s in sloc.keys():
    df_ = df.loc[df.Sloc==s].groupby(['Year', 'Month', 'Day']).sum().reset_index()
    df_['Price_Daily'] = df_['Purchase_price_thb_ton']/df_['MillWeight']
    df_.drop('Price_ton', axis=1 ,inplace=True)
    wp[s] = df_.to_json(orient = 'records')

In [ ]:
import json
with open('Sloc.json', 'w') as o: json.dump(sloc, o)
with open('Zone.json', 'w') as o: json.dump(zone, o)
with open('wp.json', 'w') as o: json.dump(wp, o)

In [ ]:
wpm = {}
with open('wp.json', 'r') as o: wp = json.load(o)
for s in sloc.keys():
    wp[s]          = pd.read_json(wp[s], orient = 'records')
    wp[s]['dt']    = pd.to_datetime(wp[s][['Year', 'Month', 'Day']])
    #wp[s].drop(['Year', 'Month', 'Day'], axis=1, inplace=True)
    wpm[s]         = wp[s].groupby(['Year','Month']).sum().reset_index().drop('Price_Daily', axis=1)
    wpm[s]['Price_Monthly'] = wpm[s]['Purchase_price_thb_ton']/wpm[s]['MillWeight']
    wpm[s]['Day']  = 1   
    wpm[s]['dt']   = pd.to_datetime(wpm[s][['Year', 'Month', 'Day']])
    wpm[s]['Sloc'] = s

# Test

In [ ]:
wp[s]

In [ ]:
wpms = {}
for s in sloc.keys():
    sen        = wpm[s][['MillWeight','Price_Monthly']].pct_change().apply(lambda x: round(x, 5))*100
    sen['sen'] = sen['MillWeight'] / sen['Price_Monthly']
    sen        = sen.replace(np.inf, 1000).replace(-np.inf, -1000).clip(-100, 100)
    wpms[s]    = sen.merge(wpm[s]['dt'], left_index=True, right_index=True)
    wpms[s]['Sloc'] = s

In [ ]:
wpmj, wpmsj = {}, {};
wpmsjz = pd.DataFrame()
for z in zone.keys():
    a, b = pd.DataFrame(), pd.DataFrame()
    for s in zone[z]:
        a = a.append(wpm[s])
        b = b.append(wpms[s])
    wpmj[z] = a
    wpmsj[z] = b
    wpmsjz = wpmsjz.append(wpmsj[z])

In [ ]:
tl = np.sort(wpmsjz.dt.unique())[::-1]
ss = wpmsjz.Sloc.unique()
dft= pd.DataFrame()
for t in tl:
    dfs = pd.DataFrame(ss, columns=['Sloc'])
    dfs['dt'] = t
    dft = dft.append(dfs)
wpmsjzm = wpmsjz.merge(dft, left_on=['dt','Sloc'], right_on=['dt','Sloc'], how='outer')


# Plot

In [ ]:
len(px.scatter(wpmj[z], x='dt', y='Price_Monthly', size='MillWeight', color='Sloc', size_max=60).data)

In [ ]:
px.line(wpms[s], x='dt', y='sen')

In [ ]:
fig = go.Scatter(x=wpm[s]['dt'].values, y=wpm[s]['Price_Monthly'].values,
                 marker=dict(size=wpm[s]['MillWeight'].values))
fig